## A study to understand how teams, players and umpires have used referrals in the World Test Championship

### Possible insights desired

**PHASE 1**:

- How effective was the DRS call in extending the survival at the crease?

- No. of referrals innings wise

- Who was the batting partner who has probably assited the most in DRS?

**PHASE 2**:

- How many recognized batsmen were left? And was there a missed opportunity due to DRS being recklessly taken earlier?**

- Missed reviews by teams: Did # of remaining reviews have a say


### Data points collected (Phase 1)
-  Match in Series 
-  Series Name to produce facets 
-  Match Venue
-  Match Date (Month_Year)
-  Over of referral
-  Innings of referral in game
- Team taking review
- Team Batting/Bowling
- Umpire at time of review
- Batsman at time of review
- Outcome of review 
- Innings wise dismissal data
- Active wicket partnership
- Innings wise referral data (scraped from match notes, needs a bit of formatting)
- Commentary of that particular referral ball


### Data points (Phase 2)

- no. of recognized batsman to come vs missed opportunities for them


### Required libraries 

In [20]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
from collections import defaultdict
import glob

### Collect all match wise referral DF into one

In [21]:
reviews_df=pd.DataFrame()

for glob in glob.glob("review_data/*.csv"):
    df=pd.read_csv(glob)
    reviews_df=pd.concat([df,reviews_df])
    
# reviews_df.to_csv('reviews.csv',index=False)

reviews_df

C:\Users\k.shridhar\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:5: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """


,Commentary,Over,Partnership_broken,Review_batsman,Review_outcome,Review_team,Review_umpire,Umpires_call,active_partnership,index,innings,match,reviews
0,"Jadeja to Holder, out Bowled!! No luck this ti...",59.5,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,West Indies 2nd innings,wi-vs-ind-2nd-test-india-tour-of-west-indies-2019,NaN
1,"Jadeja to Holder, FOUR, lines up for a big hea...",59.4,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,West Indies 2nd innings,wi-vs-ind-2nd-test-india-tour-of-west-indies-2019,NaN
2,"Jadeja to Holder, no run, a back foot punch to...",59.3,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,West Indies 2nd innings,wi-vs-ind-2nd-test-india-tour-of-west-indies-2019,NaN
3,"Jadeja to Holder, no run, goes back and across...",59.2,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,West Indies 2nd innings,wi-vs-ind-2nd-test-india-tour-of-west-indies-2019,NaN
4,"Jadeja to Holder, no run, the intentions are w...",59.1,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,West Indies 2nd innings,wi-vs-ind-2nd-test-india-tour-of-west-indies-2019,NaN
5,"Shami to Gabriel, no run, full and seeking the...",58.6,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,West Indies 2nd innings,wi-vs-ind-2nd-test-india-tour-of-west-indies-2019,NaN
6,"Shami to Roach, out Caught by Pant!! India, ju...",58.5,NaN,NaN,NaN,NaN,NaN,NaN,9.0,NaN,West Indies 2nd innings,wi-vs-ind-2nd-test-india-tour-of-west-indies-2019,NaN
7,"Shami to Roach, no run, pushed with soft hands...",58.4,NaN,NaN,NaN,NaN,NaN,NaN,9.0,NaN,West Indies 2nd innings,wi-vs-ind-2nd-test-india-tour-of-west-indies-2019,NaN
8,"Shami to Roach, no run, whoopsy! That's clonke...",58.3,NaN,NaN,NaN,NaN,NaN,NaN,9.0,NaN,West Indies 2nd innings,wi-vs-ind-2nd-test-india-tour-of-west-indies-2019,NaN
9,"Shami to Roach, no run, good length shaping in...",58.2,NaN,NaN,NaN,NaN,NaN,NaN,9.0,NaN,West Indies 2nd innings,wi-vs-ind-2nd-test-india-tour-of-west-indies-2019,NaN


In [ ]:
# reviews_df=pd.read_csv('review_data/reviews.csv')
reviews_df['innings_action']=(reviews_df['Review_team'].astype('str')).apply(lambda x: "Batting" if "Batting" in x else "Bowling")

### Process review events- to add overturned, review loss, 1st review loss, 2nd review loss logics

In [ ]:
df=reviews_df.loc[(~pd.isnull(reviews_df.reviews))]

# df=df[df.match=='eng-vs-aus-3rd-test-the-ashes-2019']
# ##Split into innings
# reviews_df['Review_team']=reviews_df['Review_team'].astype('str')
# df=reviews_df
# df.fillna('',inplace=True)
##Keep start review count =2
## Overturn event logic. bowling-upheld and p.broken=True [OR] batting-upheld with p.broken=False 
##1st review lost at =
##2nd review lost at = 
## Review loss logic is take innings, take team batting- check upheld or struck down or struck down umpires call. add 1 if struck down
## for team bowling- check upheld or struck down or stuck down umpires call. if struck down, then add 1.

In [ ]:
df['overturned']=df[['Review_team','Review_outcome','Partnership_broken']].apply(lambda x: True if 
(('Bowling' in x.Review_team and x.Review_outcome=='Upheld' and x.Partnership_broken==True) 
 or ('Batting' in x.Review_team and x.Review_outcome=='Upheld' and x.Partnership_broken==False)) else False,axis=1)

In [ ]:
df['review_lost']=df[['Review_team','Review_outcome','Partnership_broken']].apply(lambda x: True if 
(('Bowling' in x.Review_team and x.Review_outcome=='Struck down' and x.Partnership_broken==False) 
 or ('Batting' in x.Review_team and x.Review_outcome=='Struck down' and x.Partnership_broken==True)) else False,axis=1)


In [ ]:
df['first_review_lost']=(df.loc[df.review_lost==True].sort_values('Over').groupby(["match","innings","innings_action"])['review_lost'].rank('first')==1)

df['second_review_lost']=(df.loc[df.review_lost==True].sort_values('Over').groupby(["match","innings","innings_action"])['review_lost'].rank('first')==2)


In [ ]:
df.fillna('',inplace=True)

In [ ]:
cols_to_add_from_df=['overturned','review_lost','first_review_lost','second_review_lost']

for col in cols_to_add_from_df:
    reviews_df[col]=df.loc[df.index,col]

In [ ]:
reviews_df.fillna('',inplace=True)

### Visualisations for blog

- Batsman who took the most reviews (intiated by batsman)
- Whom did teams take a review against the most? 
- Umpire effectiveness (# of overturned decisions/total number of DRS calls for an umpire) Bar chart
- Review effectiveness by teams overall (Who had most partnership breaks when overturned)

In [ ]:
df=reviews_df[~pd.isnull(reviews_df.reviews)]

df.shape

print(df.columns)

### Overturned decisions per match

In [ ]:
overturned_per_match=df.groupby('match')['overturned'].sum()

### Total decisions per match

In [ ]:
total_per_match=df.groupby('match')['overturned'].count()

In [ ]:
ot_decisions=[]

tot_avg_decisions=[]

In [ ]:
for r in range((df.shape[0])):
    ot_decisions.append(overturned_per_match.loc[df.match.iloc[r]])
    tot_avg_decisions.append(overturned_per_match.loc[df.match.iloc[r]]/total_per_match.loc[df.match.iloc[r]])

In [ ]:
df['overturned_decisions']=ot_decisions

In [ ]:
df['ratio_overturned']=tot_avg_decisions

In [ ]:
# fig = px.bar(df,x='match',y='overturned_decisions')
# # fig.update_xaxes(categoryorder='sum descending')

# fig.show()

In [ ]:
df['team']=df['Review_team'].apply(lambda x:x.split('(')[0].strip())

df['bowler']=df['Commentary'].apply(lambda x:x.split('to')[0].strip())

In [ ]:
df.head()

In [ ]:
df.shape